### 2. 
Классификатор C4.5 и три его модификации: с оптимизацией гиперпараметра m, гиперпараметра cf и с одновременной оптимизацией обоих гиперпараметров. Эти четыре классификатора сравнивались на 14 наборах данных. На каждом датасете был посчитан AUC каждого классификатора. Данные записаны в файле:
AUCs.txt
Используя критерий знаковых рангов, проведите попарное сравнение каждого классификатора с каждым. Выберите два классификатора, различие между которыми наиболее статистически значимо.

### 5.
Сравнивая 4 классификатора между собой, мы проверили 6 гипотез. Давайте сделаем поправку на множественную проверку. Начнём с метода Холма. Сколько гипотез можно отвергнуть на уровне значимости 0.05 после поправки этим методом?

### 6. 
Сколько гипотез можно отвергнуть на уровне значимости 0.05 после поправки методом Бенджамини-Хохберга?

In [44]:
import pandas as pd
import numpy as np

from scipy.stats import pearsonr
from statsmodels.sandbox.stats.multicomp import multipletests 
from scipy import stats

In [45]:
data = pd.read_csv('AUCs.txt', sep = '\t')
data

,Unnamed: 0,C4.5,C4.5+m,C4.5+cf,C4.5+m+cf
0,adult (sample),0.763,0.768,0.771,0.798
1,breast cancer,0.599,0.591,0.590,0.569
2,breast cancer wisconsin,0.954,0.971,0.968,0.967
3,cmc,0.628,0.661,0.654,0.657
4,ionosphere,0.882,0.888,0.886,0.898
5,iris,0.936,0.931,0.916,0.931
6,liver disorders,0.661,0.668,0.609,0.685
7,lung cancer,0.583,0.583,0.563,0.625
8,lymphography,0.775,0.838,0.866,0.875
9,mushroom,1.000,1.000,1.000,1.000


In [46]:
data.shape

(14, 5)

In [47]:
data.describe()

,C4.5,C4.5+m,C4.5+cf,C4.5+m+cf
count,14.000000,14.000000,14.000000,14.000000
mean,0.804929,0.820429,0.808786,0.827214
std,0.160187,0.158583,0.167566,0.154548
min,0.583000,0.583000,0.563000,0.569000
25%,0.636250,0.666500,0.624000,0.673000
50%,0.828500,0.863000,0.876000,0.886500
75%,0.950500,0.968750,0.960250,0.965750
max,1.000000,1.000000,1.000000,1.000000


In [48]:
d = [data.iloc[:, 1].values, data.iloc[:, 2].values, data.iloc[:, 3].values, data.iloc[:, 4].values]

rank_data = []

for i in xrange(len(d) - 1):
    for j in xrange(i+1, len(d)):
        stat, p = stats.wilcoxon(d[i], d[j])
        rank_data.append([data.columns[i+1], data.columns[j+1], stat, p])
        
rank_df = pd.DataFrame.from_records(rank_data)
rank_df.columns = ['alg1', 'alg2', 'stat', 'p']
        
rank_df

,alg1,alg2,stat,p
0,C4.5,C4.5+m,6.5,0.010757
1,C4.5,C4.5+cf,43.0,0.861262
2,C4.5,C4.5+m+cf,11.0,0.015874
3,C4.5+m,C4.5+cf,18.0,0.054329
4,C4.5+m,C4.5+m+cf,22.0,0.327826
5,C4.5+cf,C4.5+m+cf,10.5,0.025314


In [49]:
(rank_df.p < 0.05).value_counts()

True     3
False    3
Name: p, dtype: int64

In [58]:
reject_holm, p_corrected_holm, a1, a2 = multipletests(rank_df.p, 
                                            alpha = 0.05, 
                                            method = 'holm')
rank_df['p_corrected_holm'] = p_corrected_holm
rank_df['reject_holm'] = reject_holm

reject_bh, p_corrected_vh, a1, a2 = multipletests(rank_df.p, 
                                            alpha = 0.05, 
                                            method = 'fdr_bh')
rank_df['p_corrected_bh'] = p_corrected_vh
rank_df['reject_bh'] = reject_bh

In [59]:
rank_df.reject_holm.value_counts()

False    6
Name: reject_holm, dtype: int64

In [60]:
rank_df.reject_bh.value_counts()

False    4
True     2
Name: reject_bh, dtype: int64

In [61]:
rank_df

,alg1,alg2,stat,p,p_corrected_holm,reject_holm,p_corrected_bh,reject_bh
0,C4.5,C4.5+m,6.5,0.010757,0.064543,False,0.047623,True
1,C4.5,C4.5+cf,43.0,0.861262,0.861262,False,0.861262,False
2,C4.5,C4.5+m+cf,11.0,0.015874,0.079372,False,0.047623,True
3,C4.5+m,C4.5+cf,18.0,0.054329,0.162986,False,0.081493,False
4,C4.5+m,C4.5+m+cf,22.0,0.327826,0.655651,False,0.393391,False
5,C4.5+cf,C4.5+m+cf,10.5,0.025314,0.101254,False,0.050627,False
